In [5]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry
import time


from lib.rai.rai_env import RaiEnv
from lib.prediction.color_seg import find_ball


In [6]:
#########################################
# initialize rai simulation here        #
#########################################
Rai = RaiEnv(tau=.01,
             realEnv="scenarios/project_env.g",
             modelEnv="scenarios/project_env.g",
             useROS=False,
             initSim=False,
             initConfig=True,
             simulatorEngine=ry.SimulatorEngine.bullet,
             verboseSim=1,
             defaultCamera=False)

#########################################
# init simulation                       #
#########################################
Rai._init_simulation()


In [38]:
Rai.run_simulation(11)




Simulation finished


In [41]:
komo = Rai.C.komo_path(2., 20, 4., True)

ball = Rai.RealWorld.getFrame("ball")

type(ball)


libry.Frame

In [7]:
komo.getPathFrames()

Rai.S.getGripperWidth

<bound method PyCapsule.getGripperWidth of <libry.Simulation object at 0x7f816804d970>>

In [40]:
type(Rai) is RaiEnv


True

In [4]:
Rai.S.getImageAndDepth()

(array([[[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
         [129, 140, 151],
         [129, 140, 151]],
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
         [129, 140, 151],
         [129, 140, 151]],
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
         [129, 140, 151],
         [129, 140, 151]],
 
        ...,
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
         [129, 140, 151],
         [129, 140, 151]],
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
         [129, 140, 151],
         [129, 140, 151]],
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
  

In [19]:
tau = 0.01
# simulate to grasp
for t in range(int(0.3 / tau)):
    time.sleep(tau)
    q = Rai.S.get_q()
    Rai.C.setJointState(q)  # set your robot model to match the real q

    Rai.S.closeGripper("R_gripper", speed=200.)

    # send velocity controls to the simulation
    Rai.S.step(np.zeros_like(q), tau, ry.ControlMode.none)
